In [14]:
import numpy as np
import pandas as pd

lookback = 5

data = pd.read_csv('monthly_prices/AAPL.csv', index_col=0)
data.head()

data_split = np.array_split(data, lookback)
df = data_split[0]
len(data.index)

60

In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

data = pd.read_csv('monthly_prices/AAPL.csv', index_col=0)

price = data[['Close']]
scaler = MinMaxScaler(feature_range=(-1, 1))
price['Close'] = scaler.fit_transform(price['Close'].values.reshape(-1,1))
price.head()

C:\Users\zpulliam\AppData\Local\Temp\ipykernel_16388\2581330153.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price['Close'] = scaler.fit_transform(price['Close'].values.reshape(-1,1))


,Close
0,-0.964926
1,-0.926232
2,-0.963659
3,-0.972780
4,-0.894560


In [3]:
from tst import TimeSeriesTransformer

## Model parameters
dim_val = 512 # This can be any value divisible by n_heads. 512 is used in the original transformer paper.
n_heads = 8 # The number of attention heads (aka parallel attention layers). dim_val must be divisible by this number
n_decoder_layers = 4 # Number of times the decoder layer is stacked in the decoder
n_encoder_layers = 4 # Number of times the encoder layer is stacked in the encoder
input_size = 1 # The number of input variables. 1 if univariate forecasting.
dec_seq_len = 92 # length of input given to decoder. Can have any integer value.
enc_seq_len = 153 # length of input given to encoder. Can have any integer value.
output_sequence_length = 58 # Length of the target sequence, i.e. how many time steps should your forecast cover
max_seq_len = enc_seq_len # What's the longest sequence the model will encounter? Used to make the positional encoder

model = TimeSeriesTransformer(
    dim_val=dim_val,
    batch_first=True,
    input_size=input_size, 
    dec_seq_len=dec_seq_len,
    out_seq_len=output_sequence_length, 
    n_decoder_layers=n_decoder_layers,
    n_encoder_layers=n_encoder_layers,
    n_heads=n_heads)

c:\Users\zpulliam\Anaconda3\envs\ds\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dataset import SP_500

dataset = SP_500('monthly', 1)
dataset.__getitem__(0)[3]['Open']

175.039993

In [53]:
import os
import numpy as np
import pandas as pd
from torch.utils.data import Dataset

class SP_500(Dataset):
    def __init__(self, splits):
        self.folder = 'test'
        self.splits = splits

        self.files = os.listdir(self.folder)
        new_files = []

        max = 0
        for file in self.files:
            data = pd.read_csv(os.path.join(self.folder, file), index_col=0)
            if len(data.index) > max:
                max = len(data.index)

        for file in self.files:
            data = pd.read_csv(os.path.join(self.folder, file), index_col=0)
            if len(data.index) == max:
                new_files.append(file)

        self.files = new_files
        self.data = []   # stores lists of csv and partition: [AAPL.csv, n]

        for file in self.files:
            for i in range(self.splits):
                self.data.append([file, i])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        file = self.data[idx][0]
        partition = self.data[idx][1]

        data = pd.read_csv(os.path.join(self.folder, file), index_col=0)
        data_split = np.array_split(data, self.splits)

        if data_split[0].shape[0] != data_split[-1].shape[0]:
            for i, sub in enumerate(data_split):
                if sub.shape[0] != data_split[-1].shape[0]:
                    data_split[i] = data_split[i][0:-1]

        data = data_split[partition]

        x = data[['Open', 'High', 'Low', 'Close', 'Volume']]
        y = data['Close']

        mins, maxs = x.min().to_numpy(), x.max().to_numpy()

        x = (x-x.min())/(x.max()-x.min())
        y = (y-y.min())/(y.max()-y.min())
        x = x.to_numpy()
        y = y.to_numpy()
        y = y.reshape(-1, 1)

        return x, y, mins, maxs

In [92]:
from torch import nn, optim

from torch.utils.data import DataLoader
from rnn import LSTM, GRU

model = LSTM(input_dim=5, hidden_dim=32, output_dim=1, num_layers=2)

dataset = SP_500(4)
dataloader = DataLoader(dataset=dataset, batch_size=2, shuffle=True, num_workers=0)

lookback = 100

for i, data in enumerate(dataloader):
    #print(data[0].shape)

    seqs = []

    for i in range(data[0].shape[1]-1 - lookback): 
        seqs.append([data[0][:,i: i + lookback, :], data[1][:, i+1: i+1 + lookback, :]])

    print(len(seqs), seqs[0][0].shape, seqs[0][1].shape) # sequences, train, target

213 torch.Size([2, 100, 5]) torch.Size([2, 100, 1])
213 torch.Size([2, 100, 5]) torch.Size([2, 100, 1])


In [81]:
x = []

x.append([data[0][:,0: 0 + 2, :], data[1][:,0: 0 + 2, :]])
x[0]

[tensor([[[1.4530e-01, 1.1948e-01, 2.1184e-01, 2.0224e-01, 1.8459e-01],
          [2.1612e-01, 3.0377e-01, 3.1479e-01, 4.2985e-01, 7.4609e-01]],
 
         [[6.0293e-04, 0.0000e+00, 0.0000e+00, 1.2620e-02, 3.6240e-01],
          [1.9144e-02, 1.1071e-02, 3.5908e-02, 1.0000e-02, 2.0667e-01]]],
        dtype=torch.float64),
 tensor([[[0.2022],
          [0.4299]],
 
         [[0.0126],
          [0.0100]]], dtype=torch.float64)]

In [82]:
print(len(data[0]))

2
